<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/2)_Consigna_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Generar una función genérica que dado un n nos permita obtener un RDD con los n-gramas del contenido de texto de wikipedia y su frecuencia***

Realizo la instalación de todo lo que utilizare en el desarrollo de la consulta.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!pip install --upgrade pandas
!pip install nltk==3.4
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

Importo librerias.

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Creo el sparkSession.

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo a mi drive para llevar acabo la lectura del archivo.

In [5]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [6]:
!pip install pyarrow

In [7]:
!pip install fastparquet

In [8]:
import pyarrow
import fastparquet

Realizo la lectura del archivo. En este caso lo leere en formato parquet, dado que al tener tantos textos, se producen errores de lectura.

In [9]:
contenido = pd.read_csv('gdrive/My Drive/TP2 - Datos/contents_text_sample.csv')
contenido.to_parquet('gdrive/My Drive/TP2 - Datos/contents_text_sample.parquet')
del contenido
cont = sqlContext.read.parquet('gdrive/My Drive/TP2 - Datos/contents_text_sample.parquet')

In [10]:
rddCont = cont.rdd

In [11]:
rddCont.take(2)

[Row(id=50, title='Anacardiaceae', text="{{Ficha de taxón\n| name = Anacardiáceas\n| image = Gui1 cashewfruit2.jpg\n| image_caption = ''[[Anacardium occidentale]]''\n| regnum = [[Plantae]]\n| divisio  = [[Magnoliophyta]]\n| classis  = [[Magnoliopsida]]\n| ordo = [[Sapindales]]\n| familia = '''Anacardiaceae'''\n| familia_authority = [[Robert Brown|R.Br.]]\n| subdivision_ranks = Géneros\n| subdivision = \nVer texto\n}}\n'''Anacardiaceae''' es una [[familia (biología)|familia]] de [[Plantae|plantas]] esencialmente [[árbol|arbóreas]] y [[arbusto|arbustivas]] perteneciente al [[Orden (biología)|orden]] [[Sapindales]]. La constituyen 77 [[género (biología)|géneros]] con unas 700 especies aceptadas, de las casi 3000 descritas, propias de países tropicales, cálidos y templados.<ref>[http://www.theplantlist.org/1.1/browse/A/Anacardiaceae/ Anacardiaceae en The Plant List, vers. 1.1, 2013]</ref>\n\n== Descripción ==\nSon árboles, arbustos, raramente subarbustos o [[trepadora]]s, frecuentemente co

Genero un listado de simbolos para filtarlos y a la hora de generar los ngramas, el word_tokenize no asuma que un simbolo(suelto) es una palabra.

In [12]:
symbols = ['!',',','"','#','$','%','&','\',''','(',')','()','*','+','-','.','/',':',';','<','=','>','?','@','[','\,',']','^','_','`','``','','{','|','}','{}','[]','~','pipe', "''", "'", '||', '==','--']

Creo la funcion obtener_ngrams, que recibe por parametro un texto y un n, que sera la cantidad de particiones, es decri si n es 3, sera un trigrama.

In [13]:
def obtener_ngrams(texto, n):
    text = texto.lower()
    text = word_tokenize(text)
    list_text = []

    for w in text:
        if w not in symbols:    #quito los simbolos sueltos
            if w[0] in symbols:
                w = w[1:]
            elif w[-1] in symbols:
                w = w[0:-1]
            list_text.append(w) 
    
    n_grams = ngrams(list_text, n)
    return n_grams

Uso el .sample para usar el 1% del rdd porque me tarda mas de hora y media ejecutar el tokenize para el rdd completo.


Creo la funcion obtenerRDD, que recibe por parametro un RDD y un n. Devuelve un RDD de ngramas, que cada cual tiene su frecuencia en el texto.

In [14]:
def obtenerRDD(rdd,n):
    rddListo = rdd.filter(lambda x: x.text != None) #Limpio el RDD de los None en los textos, que es la parte del RDD con la que trabajare.
    rddListo = rddListo.sample(False,0.01).flatMap(lambda x: [y for y in obtener_ngrams(x.text,n)]) #obtener_ngrams recibe el texto y el n para los n-grams
    rddListo = rddListo.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y) #realizo en primer lugar el map, para que me quede una estructura por ngrama del estilo ((ngrama),1),
                                                                          # y luego calculo la frecuencia del ngrama en el texto con el reduceByKey
    return rddListo

Calculo el RDD de ngramas para el rddCont (rdd del contents_text_sample). En este caso le indicio que n es igual a 5.

In [15]:
rddNgrams = obtenerRDD(rddCont, n=5)

Verifico que lo me devuelve mi funcion es del tipo RDD.

In [16]:
type(rddNgrams)

pyspark.rdd.PipelinedRDD

***Resultado***

In [17]:
rddNgrams.takeOrdered(10,lambda x: -x[1])

[(('bgcolor=', 'bgcolor=', 'bgcolor=', 'bgcolor=', 'bgcolor='), 387),
 (('censo', 'de', 'los', 'estados', 'unidos'), 323),
 (('style', 'text-align', 'center', 'top', 'rowspan=2'), 169),
 (('de', 'los', 'estados', 'unidos', 'de'), 155),
 (('ref', 'cita', 'web', 'url', 'http'), 155),
 (('raza', 'censo', 'de', 'los', 'estados'), 154),
 (('de', '2010', 'idioma', 'inglés', 'ref'), 143),
 (('editor', 'centro', 'nacional', 'de', 'huracanes'), 134),
 (('centro', 'nacional', 'de', 'huracanes', 'autor'), 134),
 (('style', 'text-align', 'center', 'top', 'rowspan=1'), 125)]

*Observación:* asumo que el word_tokenize esta tomando por ejemplo "|nombre" como una unica palabra. Y que el problema de los simbolos sueltos esta resultado, dado que ya no hay simbolos sueltos en los ngramas, como si me habian quedado en un primer momento.

He leido la documentación del tokenize y no he encontrado ninguna alternativa para "filtrar" los simbolos del texto. Recurri a otros métodos particulares, como el split, y tampoco funciono. Tambien probe realizando .replace() por palabra, pero el costo computacional se hacia muy alto. Entre otras opciones que probe para solucionar esto. 

Por ultimo, realice un analisis de los datos, y al notar que en la mayoria de los casos, los simbolos que no tenian sentido que esten siendo considerados como parte de una palabra se encontraban o al principio o al final de esta, opte por trabajar con unicamente la primer y ultima letra de cada palabra, para ahorrarme de esta forma, recorrer tooodas las letras de cada palabra del texto.